

## 1. 背景与问题

数据加载器 `FusedDataset` 会加载一个样本的全部可用数据（所有帧和轨迹点），然后通过填充或截断的方式将序列统一到固定长度，再送入模型进行训练。
然而，在实际的应用中，数据是逐点、依次到达的。我们的目标是让模型在每个时间点，仅利用当前和过去的信息，尽可能早地识别出目标类别。这两种模式之间存在偏差：
**信息泄露**：离线训练模式让模型在训练时看到了“未来”的数据，这可能导致它学习到依赖序列后期信息的“捷径”，从而在面对只有早期数据的真实推理场景时表现不佳。
**目标不一致**：训练目标（分类完整片段）与应用目标（尽早识别）不完全一致。

## 2. 解决方案：基于前缀的流式训练策略
为提升模型在流式推理任务中的泛化能力，引入了一种基于前缀采样的训练策略。具体做法是：在训练过程中，模型仅接收输入序列的随机长度前缀，以此模拟实际部署环境中“未来信息不可用”的约束条件。
该方法能够有效缓解训练推理不一致（train-test mismatch）的问题，并显著提升模型在早期阶段的预测质量。

## 3. 代码修改建议

创建一个新的数据集类 `FusedDatasetCausal`。核心改动在 `__getitem__` 方法中。


In [ ]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset
from dataclasses import dataclass

# 从 fusion/utils/dataset.py 中的基础类定义
@dataclass
class BatchFile:
    """批次文件信息"""
    batch_num: int           # 航迹批号
    label: int              # 目标类型标签
    raw_file: str          # 原始回波文件路径
    point_file: str        # 点迹文件路径
    track_file: str        # 航迹文件路径

class OriginalFusedDataset(Dataset):
    """这是原始的 FusedDataset 实现，作为对比。"""
    def __init__(self, batch_files: list[BatchFile], image_seq_len=64, track_seq_len=20):
        super().__init__()
        self.batch_files = batch_files
        self.image_seq_len = image_seq_len
        self.track_seq_len = track_seq_len

    def __len__(self):
        return len(self.batch_files)

    def _load_data(self, batch_file):
        # 这是一个模拟函数，实际应调用原有的数据加载逻辑
        # 假设它返回了变长的图像和轨迹序列
        # 随机生成一个长度，模拟真实数据的可变长度
        seq_len = random.randint(10, 100) 
        print(f"(模拟) 加载了批号 {batch_file.batch_num}，原始序列长度为: {seq_len}")
        images = torch.randn(seq_len, 1, 32, 544) # (T, C, H, W)
        tracks = torch.randn(seq_len, 15) # (T, F)
        return images, tracks

    def __getitem__(self, item):
        batch_file = self.batch_files[item]
        cls = batch_file.label - 1
        
        # 加载完整的序列
        images, tracks = self._load_data(batch_file)
        # 演示，直接返回完整序列
        return images, tracks, cls

### 3.1 修改后的 `FusedDatasetCausal` 实现

以下是修改后的版本。关键改动在 `__getitem__` 方法中：它在加载了完整数据后，会随机选择一个切片点 `t`，并只返回序列的前 `t` 个元素。

In [ ]:
class FusedDatasetCausal(Dataset):
    def __init__(self, batch_files: list[BatchFile], image_seq_len=64, track_seq_len=20):
        super().__init__()
        self.batch_files = batch_files
        self.image_seq_len = image_seq_len
        self.track_seq_len = track_seq_len

    def __len__(self):
        return len(self.batch_files)

    def _load_data(self, batch_file):
        # 这是一个模拟函数，与上面相同
        seq_len = random.randint(10, 100)
        print(f"(模拟) 加载了批号 {batch_file.batch_num}，原始序列长度为: {seq_len}")
        images = torch.randn(seq_len, 1, 32, 544)
        tracks = torch.randn(seq_len, 15)
        return images, tracks

    def __getitem__(self, item):
        batch_file = self.batch_files[item]
        cls = batch_file.label - 1

        # 1. 加载完整的序列
        images, tracks = self._load_data(batch_file)
        original_len = images.shape[0]

        # 2. 随机选择一个切片点 t (至少为1)
        slice_point_t = random.randint(1, original_len)
        print(f"  -> 应用因果训练: 随机截取前 {slice_point_t} 帧数据进行训练。")

        # 3. 截取增量序列
        images_sliced = images[:slice_point_t]
        tracks_sliced = tracks[:slice_point_t]

        # 4. 返回截取后的序列和最终的标签
        # collate_fn 之后会负责将这个可变长度的短序列填充到固定长度
        return images_sliced, tracks_sliced, cls

## 4. 用法演示
实际使用时，只需在 `train.py` 中将 `dataset.FusedDataset` 替换为 `dataset.FusedDatasetCausal` 即可。

In [ ]:
# 创建一些虚拟的批次文件用于演示
dummy_batch_files = [
    BatchFile(batch_num=101, label=1, raw_file="", point_file="", track_file=""),
    BatchFile(batch_num=102, label=2, raw_file="", point_file="", track_file=""),
]

# 实例化新的数据集
causal_dataset = FusedDatasetCausal(dummy_batch_files)

# 获取一个样本来观察效果
print("--- 获取第一个样本 ---")
images, tracks, label = causal_dataset[0]
print(f"返回的图像序列形状: {images.shape}")
print(f"返回的轨迹序列形状: {tracks.shape}")
print(f"返回的标签: {label}")

print("\n--- 获取第二个样本 ---")
images, tracks, label = causal_dataset[1]
print(f"返回的图像序列形状: {images.shape}")
print(f"返回的轨迹序列形状: {tracks.shape}")
print(f"返回的标签: {label}")